<a href="https://colab.research.google.com/github/GustavoFeitosa/Teste-de-Conhecimento/blob/main/LabGFF_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# BLOCO 1: SETUP INICIAL E FUNÇÕES AUXILIARES
# Instalações, Imports, Configuração da API, Inicialização do Modelo,
# Definição de funções reutilizáveis. Executar UMA VEZ por sessão.
# =============================================================================

# Instala as bibliotecas necessárias (garante que estão instaladas)
%pip install -q google-generativeai pdfplumber

# Imports gerais
import os
import re # Para as expressões regulares
from datetime import date
import textwrap # Para formatar melhor a saída de texto
import pdfplumber # Para extrair texto do PDF

# Imports para interagir com o Google Gemini e Google Colab
from google.colab import userdata
from google.colab import files # Para upload de arquivos
import google.generativeai as genai
from IPython.display import display, Markdown, HTML # Para exibir saídas formatadas

import warnings
warnings.filterwarnings("ignore") # Suprime alguns avisos comuns

print("Instalações e imports concluídos.")

# --- Configura a chave da API Google GenAI ---
# É recomendado armazenar a chave em Secrets no Google Colab
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("AVISO: Chave GOOGLE_API_KEY não encontrada nos Secrets do Colab.")
        print("Por favor, adicione sua chave API lá para usar os modelos GenAI.")
        GOOGLE_API_KEY = None
except userdata.notebook.NoUserdataError:
    print("AVISO: Não foi possível acessar os Secrets do Colab. Execute em um ambiente Colab autenticado.")
    GOOGLE_API_KEY = None # Define como None se não conseguir acessar secrets

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave de API configurada.")
else:
     print("Chave de API não configurada. Não será possível usar os modelos GenAI.")

# --- Inicializa o modelo GenerativeModel ---
# A inicialização só faz sentido se a API Key estiver configurada
model = None
if GOOGLE_API_KEY:
    try:
        # Usando gemini-1.5-flash-latest, que é rápido e bom para raciocínio e longos contextos.
        # gemini-pro também pode ser usado.
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        print(f"Modelo '{model.model_name}' inicializado com sucesso.")
    except Exception as e:
        print(f"ERRO ao inicializar o modelo: {e}")
        print("Verifique se a sua GOOGLE_API_KEY está correta e se o nome do modelo está disponível.")
        model = None
else:
    print("Modelo não inicializado devido à falta da API Key.")


# --- Função de Extração e Anonimização do PDF ---
def extrair_texto_anonimizado(caminho_pdf):
    """
    Extrai texto de um PDF, filtra linhas com palavras proibidas e anonimiza padrões comuns.
    Retorna o texto anonimizado.
    """
    texto_total = ''

    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            for pagina in pdf.pages:
                texto_total += pagina.extract_text() + '\n'
    except FileNotFoundError:
        print(f"ERRO: Arquivo PDF não encontrado no caminho: {caminho_pdf}")
        return None
    except Exception as e:
        print(f"ERRO ao ler o PDF: {e}")
        return None

    linhas = texto_total.split('\n')
    texto_filtrado = []

    palavras_proibidas = [
        'médico', 'cpf', 'dr(a)', 'respons', 'crbm', 'assinado', 'registro', 'crf', 'pront', 'técnico', 'dr.', 'os.', 'www.', 'fili',
        'pac', 'rg', 'nome', 'endereço', 'cnes', 'crm', 'assinatura', 'convênio', '.com', 'sr (a)', 'o.s', 'rua', 'av.',
        'nascimento', 'ficha', 'cep', 'cliente'
    ]

    for linha in linhas:
        linha_original = linha.strip()
        if any(palavra in linha_original.lower() for palavra in palavras_proibidas):
            continue

        linha_limpa = re.sub(r"\(?\d{2}\)?\s?\d{4,5}-?\d{4}", "[TELEFONE]", linha_original)
        linha_limpa = re.sub(r'\S+@\S+', '[EMAIL]', linha_limpa)
        linha_limpa = re.sub(r'(http|https)://\S+', '[SITE]', linha_limpa)
        linha_limpa = re.sub(r'\d{3}\.\d{3}\.\d{3}-\d{2}', '[CPF]', linha_limpa)
        linha_limpa = re.sub(r'\d{11,}', '[ID]', linha_limpa)
        linha_limpa = re.sub(r'\d{2}/\d{2}/\d{4}', '[DATA]', linha_limpa)
        linha_limpa = re.sub(r'\d{4}-\d{2}-\d{2}', '[DATA]', linha_limpa)

        texto_filtrado.append(linha_limpa)

    texto_anonimo = '\n'.join(texto_filtrado)
    return texto_anonimo

print("Função de extração e anonimização definida.")

# --- Função auxiliar para detectar alteração simples (reutilizada nos prompts) ---
def detectar_alteracao_simples(valor_str, ref_str):
    """Tenta detectar se um valor está fora de uma faixa de referência simples."""
    try:
        valor_str_clean = valor_str.replace(',', '.').strip()
        ref_str_clean = ref_str.replace(',', '.').strip()
        valor = float(valor_str_clean)

        if 'a' in ref_str_clean:
            partes_ref = ref_str_clean.split('a')
            if len(partes_ref) == 2:
                 low = float(partes_ref[0].strip())
                 high = float(partes_ref[1].strip())
                 if valor < low: return " (Baixo)"
                 if valor > high: return " (Alto)"
        elif '-' in ref_str_clean:
             partes_ref = ref_str_clean.split('-')
             if len(partes_ref) == 2:
                  low = float(partes_ref[0].strip())
                  high = float(partes_ref[1].strip())
                  if valor < low: return " (Baixo)"
                  if valor > high: return " (Alto)"
        elif ref_str_clean.startswith('<'):
             threshold = float(ref_str_clean[1:].strip())
             if valor >= threshold: return " (Alto)"
        elif ref_str_clean.startswith('>'):
             threshold = float(ref_str_clean[1:].strip())
             if valor <= threshold: return " (Baixo)"

        return ""
    except:
        return ""

print("Função de detecção de alteração definida.")


# --- Função auxiliar para exibir o relatório com botão de cópia ---
def display_report_with_copy_button(report_text):
    """Gera e exibe HTML/JS para mostrar um texto com um botão de cópia."""
    import random
    from IPython.display import display, HTML

    element_id = f"report-{random.randint(1000, 9999)}"

    html_content = f"""
    <div id="container-{element_id}" style="margin-top: 10px; padding: 10px; border: 1px solid #e0e0e0; border-radius: 5px;">
      <pre id="text-{element_id}" style="white-space: pre-wrap; word-wrap: break-word; max-height: 300px; overflow-y: auto; margin-bottom: 10px; padding: 5px; border: 1px solid #f0f0f0; background-color: #f9f9f9; border-radius: 4px;">{report_text}</pre>
      <textarea id="textarea-{element_id}" style="position: absolute; left: -9999px;"></textarea> <button onclick="copyToClipboard('{element_id}')"
              style="padding: 8px 15px; background-color: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer; font-size: 1em;">
        Copiar Relatório para Prontuário
      </button>
      <span id="feedback-{element_id}" style="margin-left: 10px; color: green;"></span>
    </div>

    <script>
      function copyToClipboard(elementId) {{ /* Chave ABERTA corrigida */
        var textElement = document.getElementById('text-' + elementId);
        var textareaElement = document.getElementById('textarea-' + elementId);
        var feedbackElement = document.getElementById('feedback-' + elementId);
        var button = document.querySelector('#container-' + elementId + ' button');

        textareaElement.value = textElement.innerText;

        navigator.clipboard.writeText(textareaElement.value).then(function() {{ /* Chave ABERTA corrigida */
          // Feedback visual de sucesso
          feedbackElement.innerText = 'Copiado!';
          button.style.backgroundColor = '#81c784'; // Um verde mais claro para feedback
          feedbackElement.style.color = 'green';
          setTimeout(function() {{ /* Chave ABERTA corrigida */
            feedbackElement.innerText = '';
             button.style.backgroundColor = '#4CAF50'; // Volta ao verde original
          }}, 2000); /* Chave FECHADA corrigida */
        }}, function(err) {{ /* Chave ABERTA corrigida */
          // Feedback visual de erro
          feedbackElement.innerText = 'Erro ao copiar!';
          button.style.backgroundColor = '#e57373'; // Um vermelho claro para erro
          feedbackElement.style.color = 'red';
          console.error('Could not copy text: ', err);
           setTimeout(function() {{ /* Chave ABERTA corrigida */
            feedbackElement.innerText = '';
             button.style.backgroundColor = '#4CAF50'; // Volta ao verde original
          }}, 2000); /* Chave FECHADA corrigida */
        }}); /* Chave FECHADA corrigida */
      }} /* Chave FECHADA corrigida */
    </script>
    <style>
     /* Estilos básicos podem ser adicionados aqui se necessário */
     #container-{element_id} button:hover {{ opacity: 0.9; }} /* Chaves ABERTA e FECHADA corrigidas */
    </style>
    """
    display(HTML(html_content))

print("Função de exibição com botão de cópia definida.")

print("\n--- Bloco 1 (Setup) concluído. Execute o Bloco 2 (Upload) em seguida. ---")

Instalações e imports concluídos.
Chave de API configurada.
Modelo 'models/gemini-1.5-flash-latest' inicializado com sucesso.
Função de extração e anonimização definida.
Função de detecção de alteração definida.
Função de exibição com botão de cópia definida.

--- Bloco 1 (Setup) concluído. Execute o Bloco 2 (Upload) em seguida. ---


In [ ]:
# =============================================================================
# BLOCO 2: CONSENTIMENTO, UPLOAD DO PDF E EXTRAÇÃO
# Inclui o pedido de consentimento LGPD antes do upload e extração.
# Armazena o resultado em 'texto_anonimo_do_pdf'. Executar UMA VEZ por PDF.
# =============================================================================

print("\n--- Bloco 2: Consentimento, Upload e Extração do PDF ---")

# --- Fluxo de Consentimento LGPD ---
display(Markdown("## Termos de Consentimento para Processamento de Exame Laboratorial (LGPD)"))
consent_text = """
Ao utilizar esta ferramenta para processar seu exame laboratorial em formato PDF, você consente com o tratamento dos seus dados de saúde, classificados como dados pessoais sensíveis pela Lei Geral de Proteção de Dados (LGPD).

**Entenda o processo:**
1. Você fará o upload de um arquivo PDF do seu exame para este ambiente seguro no Google Colab.
2. O sistema irá extrair o texto do PDF.
3. Uma função automática tentará anonimizar informações que possam identificá-lo diretamente (como nome, CPF, telefones, etc.).
4. O **texto anonimizado** será processado por um modelo de Inteligência Artificial (Google Gemini) via API para gerar um resumo para seu médico, orientações dietéticas gerais e sugestões de acompanhamento com especialistas.
5. **Sua identidade não será repassada** ao modelo de IA. O processamento pela IA é feito **somente sobre o texto anonimizado**.
6. As sugestões geradas pela IA **NÃO substituem** a consulta médica e **NÃO são aconselhamento médico**. Elas devem **SEMPRE** ser discutidas, validadas e interpretadas no contexto clínico pelo seu médico assistente ou profissional de saúde qualificado.
7. O PDF original e o texto extraído/anonimizado não serão armazenados permanentemente por esta aplicação após o processamento. O Google trata os dados enviados via API conforme sua política (geralmente retenção temporária para resposta).

Ao prosseguir com o upload, você declara ter lido e consentido com este processo, sob a base legal de consentimento conforme a LGPD para a finalidade específica descrita.

Para prosseguir, digite **SIM** abaixo e pressione Enter. Para cancelar e não processar o exame, digite qualquer outra coisa.
"""
display(Markdown(consent_text))

# Solicita o consentimento do usuário
consent_response = input("Digite 'SIM' para consentir e prosseguir: ")

texto_anonimo_do_pdf = None # Inicializa a variável para garantir que ela exista

if consent_response.upper() == 'SIM':
    print("\nConsentimento recebido. Prosseguindo com o upload e extração.")
    try:
        # Garante que as funções e o modelo do Bloco 1 foram definidos
        if 'extrair_texto_anonimizado' not in locals() or model is None:
             print("ERRO: O Bloco 1 (Setup) precisa ser executado primeiro e o modelo inicializado com sucesso.")
             print("Não é possível prosseguir sem o setup.")
        else:
            # --- Processo de Upload e Extração (Mesmo de antes) ---
            uploaded = files.upload() # Interface de upload manual

            if uploaded:
                caminho_do_seu_pdf = list(uploaded.keys())[0]
                print(f"Arquivo '{caminho_do_seu_pdf}' carregado com sucesso.")

                print(f"\nIniciando extração e anonimização do PDF: {caminho_do_seu_pdf}")
                texto_anonimo_do_pdf = extrair_texto_anonimizado(caminho_do_seu_pdf) # Chama a função do Bloco 1

                if texto_anonimo_do_pdf:
                    print("✅ Extração e anonimização concluídas.")
                    print("\n--- Conteúdo do PDF (Anonimizado, Primeiras 1000 chars para debug) ---")
                    # Imprime uma parte do texto para verificação rápida (ou imprima tudo se preferir para debug completo)
                    print(textwrap.shorten(texto_anonimo_do_pdf, width=1000, placeholder="... [resto do texto] ..."))
                    print("\n--------------------------------------------------------------")
                else:
                    print("❌ Falha na extração ou anonimização do PDF.")

            else:
                print("Nenhum arquivo foi carregado.")
                caminho_do_seu_pdf = None
                texto_anonimo_do_pdf = None # Garante None se upload vazio

    except Exception as e:
        print(f"ERRO durante o upload ou extração: {e}")
        print("Certifique-se de estar em um ambiente Google Colab e ter selecionado um arquivo válido.")
        texto_anonimo_do_pdf = None # Garante None em caso de erro

else:
    print("\nConsentimento negado. O processamento do exame foi cancelado.")
    # texto_anonimo_do_pdf já é None


# Mensagem final do Bloco 2
if texto_anonimo_do_pdf:
    print("\n--- Bloco 2 (Consentimento, Upload e Extração) concluído com sucesso. Execute o Bloco 3 (Pipeline) em seguida. ---")
else:
    print("\n--- Bloco 2 (Consentimento, Upload e Extração) concluído sem extração bem-sucedida. Corrija os erros ou consinta para executar o Bloco 3. ---")


--- Bloco 2: Consentimento, Upload e Extração do PDF ---


## Termos de Consentimento para Processamento de Exame Laboratorial (LGPD)


Ao utilizar esta ferramenta para processar seu exame laboratorial em formato PDF, você consente com o tratamento dos seus dados de saúde, classificados como dados pessoais sensíveis pela Lei Geral de Proteção de Dados (LGPD).

**Entenda o processo:**
1. Você fará o upload de um arquivo PDF do seu exame para este ambiente seguro no Google Colab.
2. O sistema irá extrair o texto do PDF.
3. Uma função automática tentará anonimizar informações que possam identificá-lo diretamente (como nome, CPF, telefones, etc.).
4. O **texto anonimizado** será processado por um modelo de Inteligência Artificial (Google Gemini) via API para gerar um resumo para seu médico, orientações dietéticas gerais e sugestões de acompanhamento com especialistas.
5. **Sua identidade não será repassada** ao modelo de IA. O processamento pela IA é feito **somente sobre o texto anonimizado**.
6. As sugestões geradas pela IA **NÃO substituem** a consulta médica e **NÃO são aconselhamento médico**. Elas devem **SEMPRE** ser discutidas, validadas e interpretadas no contexto clínico pelo seu médico assistente ou profissional de saúde qualificado.
7. O PDF original e o texto extraído/anonimizado não serão armazenados permanentemente por esta aplicação após o processamento. O Google trata os dados enviados via API conforme sua política (geralmente retenção temporária para resposta).

Ao prosseguir com o upload, você declara ter lido e consentido com este processo, sob a base legal de consentimento conforme a LGPD para a finalidade específica descrita.

Para prosseguir, digite **SIM** abaixo e pressione Enter. Para cancelar e não processar o exame, digite qualquer outra coisa.


Digite 'SIM' para consentir e prosseguir: sim

Consentimento recebido. Prosseguindo com o upload e extração.


Saving sangue_ksksksksk.pdf to sangue_ksksksksk (5).pdf
Arquivo 'sangue_ksksksksk (5).pdf' carregado com sucesso.

Iniciando extração e anonimização do PDF: sangue_ksksksksk (5).pdf


✅ Extração e anonimização concluídas.

--- Conteúdo do PDF (Anonimizado, Primeiras 1000 chars para debug) ---
Laboratório de Análises Clínicas __________________________________________________________________________________ __________________________________________________________________________________ HEMOGRAMA ERITROGRAMA Val. Encontrados Valores de Referência Hemoglobina 18,6 g/dl 13.0 a 17.0 g/dl Hematócrito: 55,9 % 36.0 a 52.0 % Hemáceas: 6,11 milhões/mm3 4.5 a 6.0 milhões/mm3 V.G.M.: 91,5 fl 80.0 a 100.0 fl H.G.M.: 30,4 pg 27.0 a 33.0 pg C.H.G.M.: 33,3 g/dl 32.0 a 36.0 g/dl R.D.W. 13,5 % 12,1 a 16,3 LEUCOGRAMA Val. Econtrados Valores de Referência % Leuc./mm³ Leuc./mm³ Contagem Global 8.060 5000 a 10000 Mielócitos: 0 0 0 Metamielócitos: 0 0 0 Bastonetes: 0 0 0 a 1000 Segmentados: 95,0 7657 1800 a 7000 Eosinófilos: 0,0 0 0 a 600 Basófilos: 0,0 0 0 a 200 Linfócitos: 2,0 161 1000 a 5000 Linf. Atípicos: 2 161 0 Monócitos: 1,0 81 80 a 1200 CONTAGEM DE PLAQUETAS: 167 mil/mm³ Métod

In [ ]:
# =============================================================================
# BLOCO 3: PIPELINE DE AGENTES E APRESENTAÇÃO FINAL
# Executa sequencialmente os agentes de Parsing, Formatação, Dieta, Especialistas
# e apresenta os resultados. Executar UMA VEZ por PDF processado (após Bloco 2).
# =============================================================================

print("\n--- Bloco 3: Iniciando Pipeline de Agentes ---")

# Verifica se as dependências essenciais dos blocos anteriores estão disponíveis
if 'texto_anonimo_do_pdf' not in locals() or not texto_anonimo_do_pdf or model is None:
    print("ERRO: As dependências do Bloco 1 e Bloco 2 não foram atendidas.")
    print("Certifique-se de ter executado os Blocos 1 e 2 com sucesso e que o PDF foi extraído.")
    # Define variáveis de resultado como None para evitar erros posteriores
    dados_exame_estruturados = None
    texto_relatorio_medico = None
    orientacoes_dieteticas = None
    sugestoes_especialistas = None
else:
    print("Dependências verificadas. Iniciando execução dos agentes.")

    # --- Inicializa variáveis de resultado ---
    dados_exame_estruturados = None
    texto_relatorio_medico = None
    orientacoes_dieteticas = None
    sugestoes_especialistas = None


    # =========================================================================
    # Agente 1: Parsing e Estruturação de Dados do Exame (LLM)
    # =========================================================================
    print("\n--- Executando Agente 1: Parsing de Dados do Exame ---")
    try:
        # Definindo o prompt para o Agente de Parsing
        prompt_agente_parsing = f"""
        Você é um assistente especializado em analisar textos de exames laboratoriais.
        Sua tarefa é extrair cuidadosamente cada resultado de exame listado no texto fornecido.
        Para cada exame, identifique e extraia as seguintes informações:
        1.  Nome do Exame (ex: HEMÁCIAS, GLICOSE, COLESTEROL TOTAL)
        2.  Valor Encontrado (o resultado obtido pelo paciente)
        3.  Valor de Referência (a faixa de valores considerados normais)
        4.  Unidade de Medida (ex: MILHÕES/mm3, g/dL, mg/dL)

        Ignore quaisquer seções do texto que não sejam resultados de exames individuais com seus valores (ex: títulos, métodos, notas gerais, cabeçalhos/rodapés residuais, seções sobre material ou método).

        Formate a saída como uma lista de objetos JSON. Cada objeto JSON deve representar um exame e ter as chaves "Exame", "Valor Encontrado", "Valor Referencial", e "Unidade". Se alguma informação estiver faltando para um exame específico, use uma string vazia ("") para o valor correspondente.

        Aqui está o texto do exame:

        {texto_anonimo_do_pdf}

        Por favor, forneça apenas a lista JSON, sem texto adicional antes ou depois.
        """

        response_parsing = model.generate_content(prompt_agente_parsing)
        texto_resposta_parsing = response_parsing.text.strip()

        if texto_resposta_parsing.startswith("```json"):
            texto_resposta_parsing = texto_resposta_parsing[len("```json"):].strip()
        if texto_resposta_parsing.endswith("```"):
            texto_resposta_parsing = texto_resposta_parsing[:-len("```")].strip()

        import json
        dados_exame_estruturados = json.loads(texto_resposta_parsing)

        print("✅ Agente 1 (Parsing) concluído. Dados estruturados extraídos.")
        # Opcional: imprimir os dados JSON para debug
        # print(json.dumps(dados_exame_estruturados, indent=2, ensure_ascii=False))

    except Exception as e:
        print(f"❌ ERRO durante a execução do Agente 1 (Parsing): {e}")
        print("Resposta bruta do modelo (para depuração):")
        try: print(response_parsing.text)
        except: print("Não foi possível obter texto da resposta.")
        dados_exame_estruturados = None


    # =========================================================================
    # Formatação do Relatório para Médicos (Python) - Usa output do Agente 1
    # =========================================================================
    if dados_exame_estruturados: # Procede apenas se o parsing foi bem sucedido
        print("\n--- Formatando Relatório Final para o Médico ---")
        try:
            texto_relatorio_medico_partes = []
            # Assume nota de referência ausente por padrão, remove se achar alguma ref
            nota_referencia_ausente_final = "Valores de referência ausentes no exame fornecido."
            referencia_encontrada_flag = False # Flag para saber se encontramos alguma ref válida

            if dados_exame_estruturados:
                for item in dados_exame_estruturados:
                    exame = item.get('Exame', 'N/A')
                    valor = item.get('Valor Encontrado', 'N/A')
                    ref = item.get('Valor Referencial', 'N/A')
                    unidade = item.get('Unidade', 'N/A')

                    # Usa a função auxiliar para detectar alteração e obter o marcador
                    marcador_alterado = detectar_alteracao_simples(valor, ref)
                    if marcador_alterado:
                         referencia_encontrada_flag = True # Se detectou alteração, a ref foi usada

                    # Monta a string para este exame no formato desejado: Nome: Valor(Marcador) Unidade
                    linha_exame_formatada = f"{exame}: {valor}{marcador_alterado} {unidade}"
                    texto_relatorio_medico_partes.append(linha_exame_formatada)

                # Junta todas as partes dos exames com '; '
                texto_relatorio_medico = "; ".join(texto_relatorio_medico_partes)

                # Adiciona a nota de referência ausente apenas se NENHUMA referência foi usada para comparação
                if not referencia_encontrada_flag:
                     texto_relatorio_medico += "\n\n" + nota_referencia_ausente_final

                # Adiciona a frase do Human in the Loop no final
                texto_relatorio_medico += "\n\nEste resumo foi gerado por um sistema de IA. Validação médica e interpretação clínica são essenciais."

                print("✅ Relatório formatado com sucesso.")

            else:
                 texto_relatorio_medico = "Não foi possível formatar o relatório: Dados estruturados estão vazios."
                 print(f"❌ Falha na formatação do relatório: {texto_relatorio_medico}")
                 texto_relatorio_medico = None # Garante que a variável é None em caso de lista vazia

        except Exception as e:
            print(f"❌ ERRO durante a formatação do relatório: {e}")
            texto_relatorio_medico = None
    else:
        print("❌ Pulando formatação do relatório: Agente 1 falhou ou não encontrou dados.")


    # =========================================================================
    # Agente 3: Orientação Dietética (LLM) - Usa output do Agente 1
    # =========================================================================
    if dados_exame_estruturados: # Procede apenas se o parsing foi bem sucedido
        print("\n--- Executando Agente 3: Orientação Dietética ---")
        try:
            # Prepara input para o prompt (mesma lógica de antes)
            dados_formatados_para_analise_dietetica = ""
            if dados_exame_estruturados:
                for item in dados_exame_estruturados:
                    exame, valor, ref, unidade = item.get('Exame', 'N/A'), item.get('Valor Encontrado', 'N/A'), item.get('Valor Referencial', 'N/A'), item.get('Unidade', 'N/A')
                    marcador_alterado = detectar_alteracao_simples(valor, ref)
                    dados_formatados_para_analise_dietetica += (
                        f"Exame: {exame}; Valor: {valor}{marcador_alterado}; Ref: {ref}; Unidade: {unidade}\n"
                    )
                if not dados_formatados_para_analise_dietetica:
                     dados_formatados_para_analise_dietetica = "Não foi possível formatar os dados dos exames para análise de sugestões."
            else:
                dados_formatados_para_analise_dietetica = "Não foram encontrados dados de exames na extração."


            # Define prompt (mesmo de antes)
            prompt_agente_orientacao_dietetica = f"""
            Você é um assistente de saúde focado em sugerir **orientações dietéticas e nutricionais** gerais com base em resultados de exames laboratoriais.
            Sua tarefa é analisar cuidadosamente os resultados de exames fornecidos e, com base neles, sugerir mudanças genéricas e baseadas em conhecimento comum em áreas de **dieta, nutrição e hidratação**.

            Analise a seguinte lista de resultados de exames (já processados de um PDF anonimizado). Note se há resultados marcados como (Alto), (Baixo) ou (Alterado):

            {dados_formatados_para_analise_dietetica}

            Identifique os resultados que justificam orientações dietéticas ou nutricionais e gere uma lista de sugestões claras e acionáveis. Seja conciso e focado nos achados mais relevantes.

            Formate as orientações como uma lista numerada ou com marcadores simples.

            **INSTRUÇÃO CRÍTICA SOBRE HIDRATAÇÃO:**
            SE (e somente se) você incluir sugestões relacionadas à hidratação (ingestão de líquidos), você DEVE incluir a seguinte ressalva imediatamente após a sugestão de hidratação ou em uma nota separada relacionada:
            'É fundamental que se alinhe a quantidade de ingestão diária de líquidos com o médico assistente, caso alguma condição de saúde do paciente imponha restrição.'

            É **fundamental** que, no final da sua resposta (após a lista de orientações e a ressalva de hidratação, se aplicável), você inclua uma linha em branco e, em seguida, uma frase clara e destacada informando que estas são orientações geradas por um sistema de IA, não constituem aconselhamento médico e devem **SEMPRE** ser discutidas e validadas por um médico ou nutricionista qualificado que tenha o contexto clínico completo do paciente.

            Siga o formato de texto puro para a lista de orientações e as frases finais de validação/ressalva. Não inclua texto adicional antes das orientações ou entre a lista e as ressalvas.

            Gere as orientações dietéticas:
            """

            response_orientacao_dietetica = model.generate_content(prompt_agente_orientacao_dietetica)
            orientacoes_dieteticas = response_orientacao_dietetica.text.strip()

            if not orientacoes_dieteticas:
                 print("AVISO: O modelo não gerou orientações dietéticas.")
                 orientacoes_dieteticas = "Não foi possível gerar orientações dietéticas com base nos resultados."

            print("✅ Agente 3 (Orientação Dietética) concluído.")

        except Exception as e:
            print(f"❌ ERRO durante a execução do Agente 3 (Orientação Dietética): {e}")
            print("Resposta bruta do modelo (para depuração):")
            try: print(response_orientacao_dietetica.text)
            except: print("Não foi possível obter texto da resposta.")
            orientacoes_dieteticas = None
    else:
        print("❌ Pulando Agente 3 (Orientação Dietética): Agente 1 falhou ou não encontrou dados.")
        orientacoes_dieteticas = "Não foi possível gerar orientações dietéticas." # Define mensagem de fallback


    # =========================================================================
    # Agente 4: Sugestões de Especialistas (LLM) - Usa output do Agente 1
    # =========================================================================
    if dados_exame_estruturados: # Procede apenas se o parsing foi bem sucedido
        print("\n--- Executando Agente 4: Sugestões de Especialistas ---")
        try:
            # Prepara input para o prompt (mesma lógica de antes)
            dados_formatados_para_analise_especialista = ""
            if dados_exame_estruturados:
                for item in dados_exame_estruturados:
                    exame, valor, ref, unidade = item.get('Exame', 'N/A'), item.get('Valor Encontrado', 'N/A'), item.get('Valor Referencial', 'N/A'), item.get('Unidade', 'N/A')
                    marcador_alterado = detectar_alteracao_simples(valor, ref)
                    dados_formatados_para_analise_especialista += (
                        f"Exame: {exame}; Valor: {valor}{marcador_alterado}; Ref: {ref}; Unidade: {unidade}\n"
                    )
                if not dados_formatados_para_analise_especialista:
                     dados_formatados_para_analise_especialista = "Não foi possível formatar os dados dos exames para análise de sugestões."
            else:
                 dados_formatados_para_analise_especialista = "Não foram encontrados dados de exames na extração."


            # Define prompt (mesmo de antes)
            prompt_agente_especialistas = f"""
            Você é um assistente de saúde focado em identificar possíveis necessidades de acompanhamento com especialistas médicos com base em resultados de exames laboratoriais.
            Sua tarefa é analisar cuidadosamente os resultados de exames fornecidos e sugerir **tipos de médicos especialistas** que poderiam ser consultados para avaliação e manejo das alterações encontradas.

            Analise a seguinte lista de resultados de exames (já processados de um PDF anonimizado). Note se há resultados marcados como (Alto), (Baixo) ou (Alterado):

            {dados_formatados_para_analise_especialista}

            Identifique os resultados ou padrões que possam indicar a necessidade de avaliação por um especialista (ex: alterações renais, hepáticas, hormonais, hematológicas, cardiovasculares, etc.).

            Gere uma lista concisa de sugestões de **tipos de especialistas** (ex: "Endocrinologista", "Nefrologista", "Cardiologista", "Hematologista", "Gastroenterologista").

            Formate as sugestões como uma lista numerada ou com marcadores simples.

            **INSTRUÇÃO CRÍTICA FINAL:** Ao final da sua resposta, em uma **linha separada**, inclua a seguinte frase **EXATA**:
            'É fundamental alinhar com o seu médico assistente a indicação ou necessidade de acompanhar adicionalmente com outros médicos especialistas conforme sugerido pela IA.'

            Siga o formato de texto puro para a lista de especialistas e a frase final. Não inclua texto adicional antes da lista ou entre a lista e a frase final.

            Gere as sugestões de especialistas:
            """

            response_especialistas = model.generate_content(prompt_agente_especialistas)
            sugestoes_especialistas = response_especialistas.text.strip()

            if not sugestoes_especialistas:
                 print("AVISO: O modelo não gerou sugestões de especialistas.")
                 sugestoes_especialistas = "Não foi possível gerar sugestões de especialistas com base nos resultados."

            print("✅ Agente 4 (Sugestões de Especialistas) concluído.")

        except Exception as e:
            print(f"❌ ERRO durante a execução do Agente 4 (Sugestões de Especialistas): {e}")
            print("Resposta bruta do modelo (para depuração):")
            try: print(response_especialistas.text)
            except: print("Não foi possível obter texto da resposta.")
            sugestoes_especialistas = None
    else:
        print("❌ Pulando Agente 4 (Sugestões de Especialistas): Agente 1 falhou ou não encontrou dados.")
        sugestoes_especialistas = "Não foi possível gerar sugestões de especialistas." # Define mensagem de fallback


    # =========================================================================
    # Apresentação Final dos Resultados
    # =========================================================================
    print("\n--- Apresentando Todos os Resultados ---")

    # Verifica quais resultados foram gerados com sucesso para exibição
    relatorio_pronto = texto_relatorio_medico is not None
    orientacoes_prontas = orientacoes_dieteticas is not None and orientacoes_dieteticas != "Não foi possível gerar orientações dietéticas."
    especialistas_prontos = sugestoes_especialistas is not None and sugestoes_especialistas != "Não foi possível gerar sugestões de especialistas."


    if relatorio_pronto or orientacoes_prontas or especialistas_prontos:
        display(Markdown("## Resultados do Processamento do Exame"))

        # --- Exibir o Relatório para o Médico ---
        display(Markdown("### Relatório Resumido para o Médico"))
        if relatorio_pronto:
            display_report_with_copy_button(texto_relatorio_medico)
        else:
            display(Markdown("> *Não foi possível gerar o relatório para o médico. Verifique as etapas anteriores.*"))


        # --- Exibir as Orientações Dietéticas ---
        display(Markdown("### Orientações Dietéticas Sugeridas"))
        if orientacoes_prontas:
            display(Markdown(orientacoes_dieteticas)) # Já contém a ressalva HITL e de hidratação
        else:
            display(Markdown("> *Não foi possível gerar as orientações dietéticas. Verifique a etapa correspondente.*"))


        # --- Exibir as Sugestões de Especialistas ---
        display(Markdown("### Sugestões de Acompanhamento com Especialistas"))
        if especialistas_prontos:
            display(Markdown(sugestoes_especialistas)) # Já contém a ressalva HITL específica
        else:
             display(Markdown("> *Não foi possível gerar as sugestões de especialistas. Verifique a etapa correspondente.*"))

        display(Markdown("---"))
        display(Markdown("✅ **Processamento do exame concluído.**"))
        # A ressalva geral já está incluída nos outputs dos agentes ou na formatação do relatório.
        # display(Markdown("Lembre-se que todas as sugestões e informações geradas pela IA devem ser discutidas e validadas com um profissional de saúde qualificado."))


    else:
        display(Markdown("## Resultados do Processamento do Exame"))
        display(Markdown("❌ **Não foi possível apresentar os resultados. Todas as etapas de geração falharam.**"))
        display(Markdown("Por favor, execute as células anteriores sequencialmente e verifique se há erros."))


print("\n--- Bloco 3 (Pipeline e Apresentação) concluído. ---")


--- Bloco 3: Iniciando Pipeline de Agentes ---
Dependências verificadas. Iniciando execução dos agentes.

--- Executando Agente 1: Parsing de Dados do Exame ---
✅ Agente 1 (Parsing) concluído. Dados estruturados extraídos.

--- Formatando Relatório Final para o Médico ---
✅ Relatório formatado com sucesso.

--- Executando Agente 3: Orientação Dietética ---
✅ Agente 3 (Orientação Dietética) concluído.

--- Executando Agente 4: Sugestões de Especialistas ---
✅ Agente 4 (Sugestões de Especialistas) concluído.

--- Apresentando Todos os Resultados ---


## Resultados do Processamento do Exame

### Relatório Resumido para o Médico

### Orientações Dietéticas Sugeridas

1.  **Controlar o consumo de alimentos ricos em ferro:** Os níveis de hemoglobina, hematócrito e hemácias estão elevados, indicando possível sobrecarga de ferro.  Reduza o consumo de carnes vermelhas, fígado, espinafre e outros alimentos ricos em ferro.

2.  **Aumentar o consumo de alimentos ricos em fibras:**  A contagem global de leucócitos está baixa e os níveis de linfócitos também estão abaixo do esperado. Uma dieta rica em fibras pode auxiliar na saúde intestinal e no sistema imunológico.  Incorpore frutas, vegetais, grãos integrais e leguminosas à sua dieta.

3.  **Avaliar a necessidade de suplementação de vitaminas e minerais:** A baixa contagem de leucócitos e linfócitos sugere a necessidade de avaliação da ingestão de vitaminas e minerais essenciais para a função imunológica.  Consulte um médico para avaliar a possibilidade de suplementação.

4.  **Monitorar a ingestão de proteínas:** Os níveis de AST e ALT estão elevados, indicando possível dano hepático.  É importante monitorar a ingestão proteica e evitar excesso, principalmente se houver problemas hepáticos preexistentes.

5.  **Hidratação adequada:**  Mantenha uma ingestão adequada de líquidos ao longo do dia. *É fundamental que se alinhe a quantidade de ingestão diária de líquidos com o médico assistente, caso alguma condição de saúde do paciente imponha restrição.*

6.  **Dieta rica em antioxidantes:** A bilirrubina total está elevada, e os níveis de AST e ALT também são altos, o que pode sugerir um estresse hepático. Uma dieta rica em frutas e vegetais ricos em antioxidantes pode auxiliar na proteção celular contra danos oxidativos.

7.  **Consultar um nutricionista:**  Dada a complexidade dos resultados e a presença de diversas alterações, é fundamental o acompanhamento de um nutricionista para elaborar um plano alimentar personalizado e seguro, considerando as particularidades do seu caso.


Estas são orientações geradas por um sistema de IA, não constituem aconselhamento médico e devem SEMPRE ser discutidas e validadas por um médico ou nutricionista qualificado que tenha o contexto clínico completo do paciente.

### Sugestões de Acompanhamento com Especialistas

1. Hematologista
2. Nefrologista
3. Hepatologista/Gastroenterologista


É fundamental alinhar com o seu médico assistente a indicação ou necessidade de acompanhar adicionalmente com outros médicos especialistas conforme sugerido pela IA.

---

✅ **Processamento do exame concluído.**


--- Bloco 3 (Pipeline e Apresentação) concluído. ---
